In [1]:
import numpy as np
import pandas as pd
import itertools

import copy
import sys, os
import datetime

from scipy.optimize import fsolve
from scipy.special import gamma


In [2]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__



In [3]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))

In [4]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [5]:
def gen_call_arrival_time():
    return np.random.exponential(4)

In [6]:
def gen_call_priority():
    return int(np.random.uniform(0,1) < 0.25)

In [7]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [8]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return (scale * np.random.weibull(shape))[0]

In [9]:
class Call(object):
    
    def __init__(self, call_index, location, time_arrival, interarrival_time, priority = 0):
        
        self.call_index = call_index
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        self.priority = priority

In [10]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [11]:
def get_ambulance_travel_time(distance, speed = 30):
    return distance/speed

In [12]:
class Ambulance(object):
    
    def __init__(self, base, speed = 30):
        
        #fix the ambulance's home base
        self.base = base
        
        #set the ambulance's travel speed
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        self.call = -1 # -1 if not assigned to any call
        
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self, speed, base = None):
        
        if base is None:
            base = self.base
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, base)
        travel_time = get_ambulance_travel_time(distance, speed)
        self.update_status(5, self.origin, self.base, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        self.call = -1
        
    def return_to_base(self):
        self.update_status(0, self.base, self.base, copy.deepcopy(self.endtime), np.inf)
        
        
    def assign_to_call(self, call, time, index, speed):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance, speed)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        self.call = call.call_index # updated assigned call index
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.pop(call.call_index)
    
    def transport_to_hospital(self, city, speed):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = np.inf
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance, speed)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [13]:
def arrival(call_index, ambulanceList, callList, time_arrival, M, timeline, speed):
    
    call = Call(call_index, gen_call_arrival_loc(city), time_arrival, 
                gen_call_arrival_time(), priority = gen_call_priority())
    
    if len(callList) >= M:
        # print("New call arrived. No more capacity. Reject call.")
        i = timeline.shape[0]
        timeline.loc[i] = [call_index, '', 7, time_arrival]
    
    else:
    
        # print("New call arrived. Add to call list.")
        callList[call_index] = call

        assign = -1
        index = 0

        nearest_distance = np.inf
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            # print("Idle ambulance at base available. Assign. Now travelling to call location.")
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, assign, 1, time_arrival]
            ambulanceList[assign].assign_to_call(call, time_arrival, assign, speed)

        # else:
            # print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival, timeline

In [14]:
def get_first(ambulance, callList, M, k):
    
    call_index = -1
    min_arrival_time = np.inf
    for call_id, call in callList.items():
        if call.status == -1:
            call_time = call.arrival_time
            if call_time < min_arrival_time:
                call_index = call_id
                min_arrival_time = call_time
        
    return call_index


In [15]:
def get_first_highpriority(ambulance, callList, M, k):
    
    # Assign based on FCFS within priority
    # assume only high and low, two levels or priority
    
    
    high_call_index = -1
    low_call_index = -1
    
    high_min_arrival_time = np.inf
    low_min_arrival_time = np.inf
    for call_id, call in callList.items():
        if call.status == -1:
            call_time = call.arrival_time
            
            if call.priority == 1:
                if call_time < high_min_arrival_time:
                    high_call_index = call_id
                    high_min_arrival_time = call_time
            
            else:
                if call_time < low_min_arrival_time:
                    low_call_index = call_id
                    low_min_arrival_time = call_time
            

    if high_call_index > -1:
        return high_call_index
    else:
        return low_call_index
    

In [16]:
def get_nearest(ambulance, callList, M, k):
    
    ambulance_loc = ambulance.origin
    
    min_distance = np.inf
    call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance < min_distance:
                call_index = call_id
                min_distance = distance
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [17]:
def get_nearest_threshold(ambulance, callList, M, k):
    # only serve unassigned calls within k distance from the ambulance
    
    ambulance_loc = ambulance.origin
    
    min_distance = k
    call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance <= min_distance:
                call_index = call_id
                min_distance = distance
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [18]:
def get_nearest_threshold_else_fcfs(ambulance, callList, M, k):
    # only serve unassigned calls within k distance from the ambulance
    # if no unassigned calls within the threshold, perform according to fcfs
    
    ambulance_loc = ambulance.origin
    
    min_distance = k
    call_index = -1
    min_arrival_time = np.inf
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance < min_distance:
                call_index = call_id
                min_distance = distance
            elif distance > k:
                if call.arrival_time < min_arrival_time:
                    call_index = call_id
                    min_arrival_time = call.arrival_time
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [19]:
def get_nearest_highpriority(ambulance, callList, M, k):
    
    ambulance_loc = ambulance.origin
    
    min_high_distance = np.inf
    min_low_distance = np.inf
    high_call_index = -1
    low_call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            
            if call.priority == 1:
                if distance < min_high_distance:
                    high_call_index = call_id
                    min_high_distance = distance
            else:
                if distance < min_low_distance:
                    low_call_index = call_id
                    min_low_distance = distance
    
    if high_call_index > -1:
        return high_call_index
    elif low_call_index > -1:
        return low_call_index
    else:
        return -1

In [20]:
def get_next_event(time_arrival, ambulanceList, callList, city, M, timeline, policy, speed, time_threshold):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    
    if next_event_time == time_arrival:
        # print("New call arrived.")
        i = timeline.shape[0]
        all_call = set(timeline['Call'])
        call_index = len(all_call) if '' not in all_call else len(all_call)-1
        timeline.loc[i] = [call_index, '', 0, time_arrival]
        ambulanceList, callList, time_arrival, timeline = arrival(call_index, ambulanceList, callList, time_arrival, M, timeline, speed)
        
    else:
        if ambulanceList[index_min].status == 1:
            # print("Now reach call location. Start at-location treatment. Remove call from call list.")
            call_index = ambulanceList[index_min].call
            call = callList[call_index]
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 2, next_event_time]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            # print("Now finish at-location treatment. Start going to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 3, next_event_time]
            ambulanceList[index_min].transport_to_hospital(city, speed)
            
        elif ambulanceList[index_min].status == 3:
            # print("Now reach hospital. Start transferring patient to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 4, next_event_time]
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            # print("Now finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 5, next_event_time]
            
            if len(callList) == 0:
                # print("Return to base.")
                ambulanceList[index_min].redployment(speed)
            else:
                # print("Call waiting. Assign to call in queue according to policy.")
                call_index = policy(ambulanceList[index_min], callList, M, time_threshold * speed/2)
                
                if call_index == -1:
                    # calls in callList have all been assigned with an ambulance, or exceed distance threshold
                    ambulanceList[index_min].redployment(speed)
                else:
                    i = timeline.shape[0]
                    timeline.loc[i] = [call_index, index_min, 1, next_event_time]
                    ambulanceList[index_min].assign_to_call(callList[call_index], next_event_time, index_min, speed)
                    
        elif ambulanceList[index_min].status == 5:
            i = timeline.shape[0]
            timeline.loc[i] = [-1, index_min, 6, next_event_time]
            # print("Now reployed ambulance reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, timeline, next_event_time

In [21]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [22]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call_index in callList:
        call = callList[call_index]
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

In [23]:
def get_unreachable_calls(time, ambulanceList, callList, delta, speed = 10):
    
    unreach_calls = 0
    
    for ambulance in ambulanceList:
        if ambulance.status == 1:
            target_call = callList[ambulance.call]
            target_threshold = target_call.arrival_time + delta
            reach_outside = ambulance.endtime > target_threshold
            unreach_calls +=1
    
    return unreach_calls
            

In [24]:
def get_ambulance_current_loc(ambulance, time):
    
    ambulance_origin = ambulance.origin
    ambulance_destination = ambulance.destination
    
    ambulance_start = ambulance.time
    ambulance_end = ambulance.endtime
    
    current_time = time
    travel_ratio = (current_time - ambulance_start) / (ambulance_end - ambulance_start)
    
    current_location = ((ambulance_destination[0] - ambulance_origin[0]) * travel_ratio + ambulance_origin[0],
                        (ambulance_destination[1] - ambulance_origin[1]) * travel_ratio + ambulance_origin[1])
    
    return current_location

In [25]:
def get_uncovered_calls(time, ambulanceList, callList, delta, city, speed = 10):
    
    uncovered_base = []
    for base in city.base_location:
        cover_calls = 0
        for ambulance in ambulanceList:
            if ambulance.status == 0 or ambulance.status == 5:
                current_location = get_ambulance_current_loc(ambulance, time)
                distance_to_base = get_distance(current_location, base)
                time_to_base = get_ambulance_travel_time(distance_to_base, speed = speed)
                
                if time_to_base < delta:
                    cover_calls += 1
                
        
        if cover_calls == 0:
            uncovered_base.append(1)
        else:
            uncovered_base.append(0)
            
    return uncovered_base
            

In [26]:
def get_jobcount(timeline):
    timediff = np.append(np.diff(timeline['Timestamp']), 0)
    timeline['timediff'] = timediff
    
    n = timeline.shape[0]
    numCalls = np.zeros(n)
    
    count = 0
    for i in range(n):
        event = timeline.iloc[i]['Event']
        if event == 0:
            count += 1
        elif event == 5 or event == 7: 
            count -= 1
            
        if count <0:
            print("hi")
            
        numCalls[i] = count
        
    numCalls[-1] = numCalls[n-2]
    timeline['numCalls'] = numCalls
    return timeline

In [27]:
def get_jobs(timeline):
    total = max(timeline['Call'])
    
    arrival = np.zeros(total+1)*np.nan
    assign = np.zeros(total+1)*np.nan
    reach = np.zeros(total+1)*np.nan
    onsite = np.zeros(total+1)*np.nan
    transfer = np.zeros(total+1)*np.nan
    finish = np.zeros(total+1)*np.nan
    
    for i in range(total + 1):
        c = timeline[timeline['Call'] == i]
        n = c.shape[0]
#         print(n)
        for index, row in c.iterrows():
            t = row['Timestamp']
            event = row['Event']
            if event == 0:
                arrival[i] = t
            elif event == 1:
                assign[i] = t if n > 1 else np.nan
            elif event == 2:
                reach[i] = t if n > 2 else np.nan
            elif event == 3:
                onsite[i] = t if n > 3 else np.nan
            elif event == 4:
                transfer[i] = t if n > 4 else np.nan
            elif event == 5:
                finish[i] = t if n > 5 else np.nan
            elif event == 7:
                finish[i] = t
#         print(n, arrival[i], assign[i], reach[i], onsite[i], transfer[i], finish[i])
        
    columns = ['arrival_time', 'assigned_time', 'reach_patient', 'finish_onsite', 'reach_hospital', 'finish']
    data = list(zip(arrival, assign, reach, onsite, transfer, finish))
    df = pd.DataFrame(data, columns=columns)
    
    df['waiting_time'] = df['assigned_time'] - df['arrival_time']
    df['total_time'] = df['finish'] - df['arrival_time']
    return df

In [28]:
def performance_metric(timeline, df, target, c = 4):
    
    t = timeline.iloc[-1]['Timestamp']
    P = timeline.groupby('numCalls')['timediff'].sum() / t
    
    expectn = sum(P * P.index)
    try:
        expectw = sum(P[c+1:] * (P.index[c+1:]-c))
    except:
        expectw = sum(P[c+1:] * (P.index[c:]-c))
        
    utilization = (expectn - expectw) / c
    
    print('\nUtilization:', utilization)
    print('Expected number of jobs in system:', expectn)
    print('Expected number of jobs in queue:', expectw)
    
    df_complete = df.dropna(axis=0)
    print('Expected time in queue:', np.mean(df_complete['waiting_time']))
    print('Expected time in system:', np.mean(df_complete['total_time']))
    print("Total completed patients: ",  len(df_complete))
    
    assigned = df[df['assigned_time'] > 0]
    print("Total assigned patients: ", len(assigned))
    count = 0
    for index, row in assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
    
    print("Total unreachable calls:", count)
    

# Set up the City

In [29]:
city_dimension = 150 # assume square
num_hospital = 2
num_base = 6
time_threshold = 9
call_list_threshold = 20
speed = 30

city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

# Heuristic Policies

In [30]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_first, speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [31]:
print((end - start).total_seconds())

22.667255


In [32]:
# enablePrint()

In [33]:
# time_arrival, ambulanceList, callList, timeline, time = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline, get_nearest_unassigned_threshold)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

Next Arrival:  5764.061172619193
Ambulance List Print: 
Status List:  [0, 2, 0, 2, 0, 3]
Origin List:  [(37.5, 50.0), (50.8854426247141, 148.85852507729146), (75.0, 50.0), (89.94588622600045, 148.87786875774742), (112.5, 50.0), (128.52250568809987, 138.81906152053105)]
Destination List:  [(37.5, 50.0), (50.8854426247141, 148.85852507729146), (75.0, 50.0), (89.94588622600045, 148.87786875774742), (112.5, 50.0), (100.0, 75.0)]
Event start time List:  [5759.080994420355, 5755.967065881709, 5756.912314270811, 5760.427657530598, 5753.633627990632, 5759.668144573247]
Event end time List:  [inf, 5785.06247130026, inf, 5773.258402912271, inf, 5762.746196813535]
Call List Print: 
Status List:  []
Location List:  []
Arrival time List:  []


In [34]:
timeline = get_jobcount(timeline)

In [35]:
df = get_jobs(timeline)

In [36]:
performance_metric(timeline, df, time_threshold, c= num_base)


Utilization: 0.7818558116110714
Expected number of jobs in system: 6.119196103296766
Expected number of jobs in queue: 1.4280612336303375
Expected time in queue: 5.830549921022461
Expected time in system: 24.265216202317806
Total completed patients:  1452
Total assigned patients:  1454
Total unreachable calls: 441


# FCFS with Priority

In [37]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

In [38]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_first_highpriority, 
                   speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [39]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)


Utilization: 0.7858148346069186
Expected number of jobs in system: 6.3252553838678836
Expected number of jobs in queue: 1.6103663762263722
Expected time in queue: 6.697772940242278
Expected time in system: 25.732234196453202
Total completed patients:  1412
Total assigned patients:  1417
Total unreachable calls: 369


# Neareset

In [40]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

In [41]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_nearest, speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [42]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)


Utilization: 0.7953867891698692
Expected number of jobs in system: 6.1387948408830715
Expected number of jobs in queue: 1.3664741058638568
Expected time in queue: 5.51787188745256
Expected time in system: 24.124704892704887
Total completed patients:  1465
Total assigned patients:  1467
Total unreachable calls: 302


# Nearest with High Priority

In [43]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

In [44]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_nearest_highpriority, 
                   speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [45]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)


Utilization: 0.7569405014656354
Expected number of jobs in system: 5.4435748616666455
Expected number of jobs in queue: 0.9019318528728328
Expected time in queue: 3.9241499343370942
Expected time in system: 22.615859571555912
Total completed patients:  1382
Total assigned patients:  1387
Total unreachable calls: 219


# Threshold supplemented with FCFS

In [46]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

In [47]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_nearest_threshold_else_fcfs, 
                   speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 


In [48]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)


Utilization: 0.8235388932541757
Expected number of jobs in system: 6.524140579806549
Expected number of jobs in queue: 1.582907220281494
Expected time in queue: 6.280878869046027
Expected time in system: 25.39022501020478
Total completed patients:  1479
Total assigned patients:  1481
Total unreachable calls: 309
